In [15]:
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
#import cartopy.crs as ccrs
import pandas as pd
import xarray as xr
import numpy as np

from __future__ import division
import os
import shutil
from subprocess import call
import itertools
import fnmatch
from calendar import monthrange
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import geopandas as gpd
import pandas as pd
### automatically refresh the buffer

%load_ext autoreload
%autoreload 2

### solve the auto-complete issue

%config Completer.use_jedi = False

### lvl 1 setups

import sys
sys.path.append('/portal1/dell/GitHub/WAYS')
sys.path.append('/portal1/dell/GitHub/sanctuary')

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

### lvl 2 setups (systerm)

import os
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import matplotlib.pyplot as plt
from statsmodels.distributions.empirical_distribution import ECDF
from numpy import hstack
from matplotlib import pyplot
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import warnings
warnings.filterwarnings('ignore')
from numpy import polyfit, poly1d
from stats import cr
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
import os
import glob
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from stats import cr
from gistool import basemap, stack_image, stack_shp, stack_hatch, stack_shp_cn, ncslice
from spatial import upscale
import statsmodels.formula.api as smf
from scipy.stats.mstats import theilslopes
from scipy.stats import kendalltau
import cartopy.crs as ccrs

import pandas as pd
import scipy.stats as scs
import matplotlib.pyplot as plt

import climate_indices as indices
from climate_indices import compute
from climate_indices import utils, indices
import geopandas as gpd
from shapely import wkt
from datetime import datetime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:


for k in range(1960,1965):  ######
    df = pd.read_csv('fd_wet_traj/lag+1/lag+1_'+str(k)+'.csv') ########
    
    df['geometry'] = df['geometry'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    lonn = gdf.geometry.x.values
    latt = gdf.geometry.y.values
    sph = gdf['Specific_Humidity'].values
    traj_loc_sp = np.stack((gdf.geometry.x.values, gdf.geometry.y.values, gdf['Specific_Humidity'].values), axis=-1)
    last_traj_number = df['traj'].iloc[-1][4:]


    domain_n = np.zeros((361,721))
    domain_m = np.zeros((361,721))
    lat_largescale= list(np.linspace(90,-90,361))
    lon_largescale= list(np.linspace(-180,180,721))
    pscf_domain_n= xr.DataArray(domain_n, coords=[lat_largescale, lon_largescale], dims=['lat','lon']) 
    pscf_domain_m= xr.DataArray(domain_m, coords=[lat_largescale, lon_largescale], dims=['lat','lon'])
    
    
    print(str(k)+"begin:", datetime.now())
    
    last_traj_number = df['traj'].iloc[-1][4:]   # how much traj you have   int(last_traj_number) int(last_traj_number)
    for i in range(int(last_traj_number)):
        
        traj_indexes = df[df['traj'] == 'traj'+str(i)].index
        for j in range(traj_indexes[0], traj_indexes[-1]):  # how much traj point in specific traj
            
            pscf_loc_n=pscf_domain_n.sel(lat=traj_loc_sp[j][1],lon=traj_loc_sp[j][0],method='nearest')
            lat_pscf_n=lat_largescale.index(pscf_loc_n.lat.values)
            lon_pscf_n=lon_largescale.index(pscf_loc_n.lon.values)
            domain_n[lat_pscf_n][lon_pscf_n]=domain_n[lat_pscf_n][lon_pscf_n]+1
            
            
            # calculate how much the traj point in specific grid of (last_traj -> path_traj) 
            if traj_loc_sp[j][2] > traj_loc_sp[traj_indexes[0]][2]:
                pscf_loc_m=pscf_domain_m.sel(lat=traj_loc_sp[j][1],lon=traj_loc_sp[j][0],method='nearest')
                lat_pscf_m=lat_largescale.index(pscf_loc_m.lat.values)
                lon_pscf_m=lon_largescale.index(pscf_loc_m.lon.values)
                domain_m[lat_pscf_m][lon_pscf_m]=domain_m[lat_pscf_m][lon_pscf_m]+1
        if i%5000 == 0:
            print(str(k)+"__"+str(i), datetime.now())
    
    print(str(k)+"end:", datetime.now())
    
    pscf_sustech = domain_m/domain_n
    ds_domain_m = xr.Dataset({'domain_m': (['time', 'lat', 'lon'], domain_m.reshape(1,361,721))},
                    coords={'lon': (['lon'], lon_largescale),
                            'lat': (['lat'], lat_largescale),
                            'time': (['time'], [k]),})
    ds_domain_m.to_netcdf('pscf_wet/lag+1/domain_m_'+str(k)+'.nc')       #######

    ds_domain_n = xr.Dataset({'domain_n': (['time', 'lat', 'lon'], domain_n.reshape(1,361,721))},
                    coords={'lon': (['lon'], lon_largescale),
                            'lat': (['lat'], lat_largescale),
                            'time': (['time'], [k]),})
    ds_domain_n.to_netcdf('pscf_wet/lag+1/domain_n_'+str(k)+'.nc')      #######
    ds_pscf_sustech = xr.Dataset({'pscf': (['time', 'lat', 'lon'], pscf_sustech.reshape(1,361,721))},
                    coords={'lon': (['lon'], lon_largescale),
                            'lat': (['lat'], lat_largescale),
                            'time': (['time'], [k]),})
    
    ds_pscf_sustech.to_netcdf('pscf_wet/lag+1/pscf_'+str(k)+'.nc')    #######
    
    condition_1 = domain_n > 3*np.nanmean(domain_n) 
    condition_2 = (1.5*np.nanmean(domain_n)  < domain_n) & (domain_n <= 3*np.nanmean(domain_n) )
    condition_3 = (np.nanmean(domain_n)  < domain_n) & (domain_n <= 1.5*np.nanmean(domain_n) )
    condition_4 = domain_n <= np.nanmean(domain_n) 
    
    pscf_sustech[condition_1] *= 1.0
    pscf_sustech[condition_2] *= 0.7
    pscf_sustech[condition_3] *= 0.4
    pscf_sustech[condition_4] *= 0.2

    ds_wpscf_sustech = xr.Dataset({'wpscf': (['time', 'lat', 'lon'], pscf_sustech.reshape(1,361,721))},
                    coords={'lon': (['lon'], lon_largescale),
                            'lat': (['lat'], lat_largescale),
                            'time': (['time'], [k]),})
    ds_wpscf_sustech.to_netcdf('pscf_wet/lag+1/wpscf_'+str(k)+'.nc')      #######